In [2]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedLineDocument
import collections
import codecs
from milvus import Milvus, IndexType, MetricType 
import jieba.posseg as pseg
import numpy as np
import re

### Prepare the Training  Data

In [125]:
org_df = pd.read_csv("./data/original_article_data.csv")

In [126]:
org_df.head()

,article_id,type,article_title,article_context
0,0,medical,汤臣倍健受困“全面战争”,编者按：本文来源创业邦专栏斑马消费，作者陈晓京。\n一进入2020年，A股第一颗雷就爆了。\...
1,1,medical,Nuance公司ACI技术推出一周年，落地后提高20%病历审查效率,雷锋网消息，近日，加拿大哈利法克斯医学中心首席信息官表示，该中心正在利用人工智能文档识别技术...
2,2,medical,阿尔法公社许四清：2020，初创公司怎么熬？谈谈我的三点看法,编者按：本文来源微信公众号阿尔法公社，作者许四清，创业邦经授权转载。\n面对新冠病毒肺炎这场...
3,3,medical,医疗AI经典审批案例：糖网AI是如何获批创新医疗器械的？,雷锋网(公众号：雷锋网(公众号：雷锋网))消息，近日，由人工智能医疗器械创新合作平台、中国信...
4,4,medical,一位美国医学社会学家看中国冠状病毒：冷静下来，停止恐慌！,编者按：本文来源微信公众号伊理雅通，作者罗伯特·巴塞洛缪，创业邦经授权转载。\n本文原作者罗...


In [413]:
org_df.type.unique()

array(['medical', 'financial', 'food', 'product'], dtype=object)

### Define a Function to Read and Preprocess Text
+ tokenize text into individual words
+ remove punctuation and number 
+ Regularly remove meaningless phrases

In [135]:
# 正则过滤
context_pattern = "\(.*? \)|（.*?）|【.*?】|《.*?》|版权声明.*|[年|月|日]|公众号|本文已标注来源和出处.*|免责声明.*|本文[经|由|来].*|雷锋网|参考资料：.*|推荐阅读：.*|来源：.*|本文（含图片）为合作媒体授权.*|[^\u4e00-\u9fa5a-zA-Z0-9]|[0-9]|编者按："
title_pattern = "[^\u4e00-\u9fa5a-zA-Z0-9]"

In [178]:
def cut_sentence(sentence):
    """
        func:切词，并对切割之后的词语进行过滤，去除停用词，保留名词，英文和自定义词库中的词，长度大于2的词
    
    """
    # 停用词加载
    stopwords_path = "./data/stop_words.txt"
    stopwords_list = [i.strip() for i in codecs.open(stopwords_path).readlines()]
    
    # 切词
    seg_list = pseg.lcut(sentence)
    seg_list = [i for i in seg_list if i.word not in stopwords_list] 
    filtered_words_list = []
    
    # 过滤 eg:[pair('昨天', 't'), pair('的', 'd'), pair('苹果', 'n'), pair('霾', 'g')]
    for seg in seg_list:
        if len(seg.word) <= 1:
            continue
        elif seg.flag == "eng":
            if len(seg.word) <= 2:
                continue
            else:
                filtered_words_list.append(seg.word)
        elif seg.flag.startswith("n"):
            filtered_words_list.append(seg.word)
        elif  seg.flag in ["x", "vn", "v", "an"]:  # 是自定一个词语或者是动名词、动词和名形词等
            filtered_words_list.append(seg.word)         
    return filtered_words_list

In [179]:
# 对文章进行正则过滤和切词
org_df["article_context_cut"] = org_df.article_context.replace(context_pattern, ' ', regex=True).apply(cut_sentence)
org_df["article_title_cut"] = org_df.article_title.replace(title_pattern, ' ', regex=True).apply(cut_sentence)
org_df.to_csv("./data/processed_article_data.csv")

In [180]:
org_df[["article_context_cut"]].apply(lambda x: [" ".join(i) for i in x]).to_csv("./data/tagged_article_data.csv", header=None, index=False)

### Training the Model
- Tagged document
- Instantiate a Doc2Vec model
- Build a vocabulary
- Train model

In [3]:
sentence = TaggedLineDocument("./data/tagged_article_data.csv")

>要训练模型，我们需要将标签/编号与训练语料库的每个文档相关联。例子中，标记只是从零开始的行号，也正好是文章id。

In [8]:
for i in sentence:
    print(i)
    break

TaggedDocument(['A股', '汤臣', '健因', '计提', '商誉', '减值', '减值', '股权', '投资', '资产', '减值', '预计', '亏损', '净利润', '下滑', '超过', '减值', '公司', '净利润', '增长', '公司', '狂飙', '突进', '行业', '环境', '变化', '投资', '失利', '公司', '主营业务', '策略', '选择', '弊端', '显现', '广告', '膳食', '营养', '补充剂', '行业', '打响', '品牌', '领域', '全面战争', '导致', '汤臣', '危机', '缠身', '收购', '标的', '业绩', '汤臣', '披露', '业绩', '预告', '公司', '归母', '净利润', '亏损', '亏损', '原因', '旗下', '公司', 'LSG', '业绩', '预期', '减值', '测试', '预计', '计提', '商誉', '减值', '计提', '减值', '参股', '企业', '股权', '投资', '计提', '资产', '减值', '提前', '公司', '归母', '净利润', '公司', '问询', '回复', '减值', '情况', '汤臣', '发行', '股份', '收购', '澳大利亚', '益生菌', '企业', 'LSG', '交易', '溢价', '公司', '商誉', '收购', 'LSG', '所赐', 'LSG', '营业', '收入', '下降', '毛利率', '下降', '费用率', '导致', '盈利', '下滑', '收购', '业绩', '公司', '解释', '实施', '打压', '中国', '代购', '公司', '参股', '企业', '上海', '凡迪', '公司', '收入', '下滑', '亏损', '现金流', '经营', '业绩', '预期', '公司', '股权', '投资', '计提', '资产', '减值', '所持', '公司', '股权', '投资', '账面', '价值', '公司', '业务', '顺风', '顺水', '盈利', '颇丰', '投资', '收购', '收购', '了拜', '旗下', '儿童', '营养', '补充剂'

实例化一个Doc2Vec模型，其向量大小为256维，并在训练语料库上迭代54次，最小字数设置为1，以丢弃出现次数很少的字。

In [185]:

model_article = Doc2Vec(vector_size=256, min_count=1, dm=1, hs=1, epochs=54, window=10)
model_article.build_vocab(sentence)
model_article.train(sentence , total_examples=model_article.corpus_count, epochs=model_article.epochs)
model_article.save("./models/doc2v_dm_hs_win_10.model")

### Assessing the Model

为了评估模型，首先为训练语料库的每个文档推断新的向量，将推断的向量与训练语料库进行比较，然后根据自相似性返回文档的排名。

In [198]:
# 测试代码 epochs = 32
rank = []
ranks = []
for row in org_df.iterrows():
    seg = row[1]["article_context_cut"]
    tag = row[0]
    infer_vec = model_article.infer_vector(seg, epochs=32)
    sims = model_article.docvecs.most_similar([infer_vec], topn=len(model_article.docvecs))
    rank = [docid for docid, sim in sims].index(tag)
    ranks.append(rank)
    
        
counter = collections.Counter(ranks)
print(counter)

Counter({0: 1507, 1: 10})


In [246]:
# 测试代码  epochs = 64
rank_64 = []
ranks_64 = []
for row in org_df.iterrows():
    seg = row[1]["article_context_cut"]
    tag = row[0]
    infer_vec = model_article.infer_vector(seg, epochs=64)
    sims = model_article.docvecs.most_similar([infer_vec], topn=10)
    rank_64 = [docid for docid, sim in sims].index(tag)
    ranks_64.append(rank_64)
    
        
counter = collections.Counter(ranks_64)
print(counter)

Counter({0: 1509, 1: 8})


基本上，超过99％的推断文档与自身最相似，大约有1％的文档错误地与另一文档最相似。针对训练矢量检查推断的矢量是一种常见的“合理性检查”，尽管不是真正的“准确性”值，但能快速的评估模型是否有效。

### insert vectors into milvus

In [201]:
# 连接milvus
milvus = Milvus()
URI = 'tcp://192.168.0.21:19530'
milvus.connect(uri =URI )

Status(code=0, message='Connected')

In [379]:
# 新建相应collection
param_doc = {'collection_name':'article_sim', 'dimension':256, 'index_file_size':1024, 'metric_type':MetricType.IP}
milvus.create_collection(param_doc)

Status(code=0, message='Create collection successfully!')

In [211]:
# 建立相应分区
for i in org_df.type.unique():
    milvus.create_partition('article_sim', i)

In [380]:
# 建立索引
IVF_SQ8_param = {'nlist': 1024}
milvus.create_index('article_sim', IndexType.IVF_SQ8, IVF_SQ8_param)

Status(code=0, message='Build index successfully!')

In [224]:
# 将向量单位化后插入到miluvs中
for row in org_df.iterrows():
    partition = row[1]["type"]
    vec = model_article.docvecs[row[0]]
    y=np.linalg.norm(vec, keepdims=True)
    z=vec/y
    milvus.insert(collection_name='article_sim', records=[z.tolist()], ids=[row[0]], partition_tag=partition)

### milvus get sim vector VS doc2vec sim

In [253]:
# 获取文章向量
partition = org_df.iloc[1,:]["type"]
article_context_cut = org_df.iloc[1,:]["article_context_cut"]
vec = model_article.infer_vector(article_context_cut)

In [412]:
print("milvus result:")
search_param = {"nprobe": 64}
y=np.linalg.norm(vec, keepdims=True)
z=vec/y
milvus.search_vectors(collection_name="article_sim", query_records=[z.tolist()], partition_tags=[partition], top_k=10, params=search_param)

milvus result:


(Status(code=0, message='Search vectors successfully!'),
 [
 [
 (id:1, distance:0.959406852722168),
 (id:153, distance:0.30066633224487305),
 (id:45, distance:0.2827969789505005),
 (id:133, distance:0.2815691828727722),
 (id:144, distance:0.2697034180164337),
 (id:148, distance:0.26817142963409424),
 (id:119, distance:0.2580987215042114),
 (id:99, distance:0.2576538920402527),
 (id:123, distance:0.25435328483581543),
 (id:46, distance:0.24800309538841248)
 ]
 ])

In [392]:
print("model result:")
model_article.docvecs.most_similar([vec], topn=10)

model result:


[(1, 0.9594069123268127),
 (153, 0.30066630244255066),
 (45, 0.2827970087528229),
 (133, 0.28156915307044983),
 (144, 0.2697034478187561),
 (148, 0.26817142963409424),
 (1201, 0.26341480016708374),
 (119, 0.2580987215042114),
 (99, 0.2576538920402527),
 (1298, 0.2552245259284973)]


### request article sim server

In [417]:
import requests
import json

In [487]:
# 测试数据
test_type1_data = {"article_id":9999,
            "status":False,
            "type":"financial",
            "article_title":"四川首富踩雷！二手车金融第一股美利车金融（ML.US）陨落，219亿贷款如何回笼？",
            "article_context":"随着美利车金融被查身为最大股东的刘永好在金融科技领域的最大一笔投资正面临更多指摘当前为数众多的美利车金融用户声称遭受套路贷短信骚扰和暴力催收由受害人自发建立的各地投诉微信群已有数十个之多贷款用户者普遍反映美利车金融APP已无法正常还款美利车营业厅已停止运营而唯一官方客服电话也已被停机在安卓商店和苹果商店美利车金融APP也被全面下架大批线上线下业务人员被警方调查问询更多业务员则在生活在惶惶不安之中数据显示截至美利车金融汽车融资贷款余额为亿元而这庞大金融贷款的背后刘永好持股且为最大股东的新网银行是美利车金融的两大重要资金方之一这种资金左右手式的合作让外界对于刘永好这种大股东身份作用充满疑惑此外据美利车数据计算最大的融资合作伙伴提供资金占比为接近成超过亿元不过这个最大融资合作伙伴究竟是谁美利车并未披露刘永好砸个亿成最大股东美利车金融被全面查封四川首富新希望集团董事长刘永好旗下的二手车金融服务平台美利车金融被调查一事正进一步演变美利车金融创始人刘雁南被警方控制作为初新希望集团在金融科技领域的最大单笔投资万美元约合亿元美利车金融几近全面崩盘据招股书披露新希望集团旗下KeyViewInvestmentsLimited持有美利车金融股权为第一大股东作为一家助贷平台美利车金融官网显示其主要提供期限为的二手车和新车购买贷款同时帮助经销商提供便捷的车辆交易库存融资服务美利车金融股票代码为ML向美国证监会SEC递交了招股书计划在美国市场IPO融资亿美元但是美利车金融并没有等来它的高光时刻这家主要靠促成汽车贷款赚钱的消费分期服务公司把车开进了内忧外患的漩涡根据知情人士提供的其与警方沟通录音证实公安部统一指挥此次抓捕行动涉及包括深圳武汉辽宁和北京在内的多个办公场所抓捕多名高管当天美利金融北京总部深圳总部武汉分部等多地均遭警方上门调查公司公告称初步确定因警方对关联公司有用分期的调查而引发对美利车金融业务的影响在国内各大网络投诉平台上对于美利车金融的投诉中也能看到套路贷非法催收等严重问题根据第三方投诉平台聚投诉上有不少针对美利车金融的投诉包括砍头息套路贷暴力催收欺诈等黑猫投诉平台上关于美利车金融的投诉也多涉及套路贷砍头息欺诈等多位用户投诉均标明美利车金融关联催收公司涉嫌爆通讯录电话骚扰短信恐吓等软暴力方式等聚投诉上一位名为李红院的用户投诉称我在美利车金融办了车贷业务不注意逾期一天有催收人员打电话过来提醒我还款合情合理但是催收人员上来就扬言限我多少分钟内把钱存到银行卡不然就让我同事知道让我家人亲戚知道并爆出我工作地址家庭地址以及家人亲戚姓名对我进行人格侮辱有消息透露目前公安方面正在调查与美利金融和刘雁南有关联的所有高炮平台寻找借款人做笔录并努力恢复底被这些高炮平台删除的系统数据实际上根据美利车金融的招股说明书来看也说明了美利车的盈利模式主要是资金掮客和催收资料显示上半美利车金融净利润亿元主要通过促成汽车贷款取得收入通常这笔助贷服务费通常占贷款本金的至而资金合作伙伴即银行收取的平均化利率为亿贷款存巨大回笼压力最大资金提供方占比近成但是这一切随着美利车金融被警方查封之后都在快速消散数据显示截至美利车金融汽车融资贷款余额为亿元这亿元存量贷款回笼压力巨大客户回款已经成为迫在眉睫的问题事实上在警方大规模带走美利车金融高层和员工之后用户开始出现拒绝还款的情况从开始这一情况越演越烈美利车唯一的官方客服电话已经无法接通在美利车金融的微信服务号美利车服务号上不到一个时间已经连续公告三次号召用户积极还款从各方透露的信息来看美利车原有的还款路径纷纷失效有知情人士分析称可能是美利车金融自身的相关金融账户或被警方查封导致还款失效多位美利车金融的贷款用户也向券商中国记者反映目前美利车金融APP已无法正常还款为避免造成被动逾期个别用户只能通过银行的还款通道协商还款而目前美利车营业厅已停止运营客服电话也无法打通在安卓商店和苹果商店美利车金融APP已经普遍下架目前美利车金融的唯一官方客服电话已经被停止使用而线下业务全面陷入瘫痪停顿大批业务人员被警方调查问询更多则在惶惶不安之中目前为数众多的美利车金融用户声称遭受套路贷骚扰和暴力催收北京广州郑州等地区由受害人自发建立的各地投诉微信群已有数十个之多正在组织联合投诉举报并以存在合同纠纷为由拒绝还款向各地方法院对美利车金融提起诉讼同时把矛头指向了为美利车金融提供资金的银行有不少车贷人表示美利车金融在他们不知情的情况下在购车贷款合同中被额外添加了一笔借款大部分车贷人都是在即将还清贷款时才发现需要多还万元甚至更多车贷人想按最初申请的贷款金额一次性还清贷款却被美利车金融一口回绝更离谱的是不少车贷人还在继续按时还款的情况下频繁接到暴力催收的骚扰电话和短信目前美利车金融在号召还款的资金方有新网银行微众银行众邦银行南京银行等其中美利车控股第一大股东刘永好旗下的新网银行是美利车金融的最重要的资金方而刘永好旗下的新希望集团是新网银行大股东持股另外新希望集团也是美利车第一大股东持股根据美利车金融招股书披露信息来看美利车金融的资金提供方集中度非常高其中最大的融资合作伙伴提供资金占比为第二名所占比例为在这两大金融机构中新网银行便是其中一家尽管信息没有披露出新网银行是第一大融资来源还是第二大融资来源根据上半美利车金融汽车融资贷款余额数据亿元来看最大的融资提供方这家银行仅对美利车一家即融出亿元市场普遍认为新希望集团同时作为新网银行和美利车金融的第一大股东新网银行成为美利车金融最重要资金合作伙伴不足为奇多位车贷用户向记者反映在美利车金融无法正常还款之后多次接到新网银行催收短信有的甚至在全额还款之后仍然没有拿到抵押的机动车登记证书不仅仅美利车金融亿元贷款回笼压力巨大事实上新网银行面临的资金出具方业务压力也非常大在全国陆续全面取缔PP网贷业务的关口新网银行仍不愿退出网贷资金存管业务甚至还坐着国内最大的PP存管银行名头且在合作中新网银行通常扮演了资金出具方的角色作为一家民营银行目前新网银行资金来源主要有三个渠道一是银行间的拆借市场新网银行已和家银行达成了同业授信和拆借协议二是联合授信已跟家银行签订了联合授信协议三是对已有的资产做成证券化来变现流转显然从这三个渠道来看很大一部分都不属于新网银行自有资金",
            "limit": 10
            }

header = {"Content-Type":"application/json"}
url = r"http://192.168.0.21:1133/sim_article/"

In [488]:
result = requests.post(url, json.dumps(test_type1_data), headers=header)
json.loads(result.text)

{'sim_article_ids': [386, 246, 637, 508, 506, 187, 550, 323, 282, 426],
 'sim_artilce_scores': [0.9829508566337826,
  0.8787390476508785,
  0.7554842286663509,
  0.7236566983959897,
  0.6926237451298618,
  0.68562444433342,
  0.6156362470315866,
  0.5495041881378759,
  0.5154631278024638,
  0.4533095727933625],
 'status': 1}

In [485]:
# 相应type召回结果
test_type2_data = {"article_id":9999,
            "status":True,
            "type":"financial",
            "article_title":"四川首富踩雷！二手车金融第一股美利车金融（ML.US）陨落，219亿贷款如何回笼？",
            "article_context":"随着美利车金融被查身为最大股东的刘永好在金融科技领域的最大一笔投资正面临更多指摘当前为数众多的美利车金融用户声称遭受套路贷短信骚扰和暴力催收由受害人自发建立的各地投诉微信群已有数十个之多贷款用户者普遍反映美利车金融APP已无法正常还款美利车营业厅已停止运营而唯一官方客服电话也已被停机在安卓商店和苹果商店美利车金融APP也被全面下架大批线上线下业务人员被警方调查问询更多业务员则在生活在惶惶不安之中数据显示截至美利车金融汽车融资贷款余额为亿元而这庞大金融贷款的背后刘永好持股且为最大股东的新网银行是美利车金融的两大重要资金方之一这种资金左右手式的合作让外界对于刘永好这种大股东身份作用充满疑惑此外据美利车数据计算最大的融资合作伙伴提供资金占比为接近成超过亿元不过这个最大融资合作伙伴究竟是谁美利车并未披露刘永好砸个亿成最大股东美利车金融被全面查封四川首富新希望集团董事长刘永好旗下的二手车金融服务平台美利车金融被调查一事正进一步演变美利车金融创始人刘雁南被警方控制作为初新希望集团在金融科技领域的最大单笔投资万美元约合亿元美利车金融几近全面崩盘据招股书披露新希望集团旗下KeyViewInvestmentsLimited持有美利车金融股权为第一大股东作为一家助贷平台美利车金融官网显示其主要提供期限为的二手车和新车购买贷款同时帮助经销商提供便捷的车辆交易库存融资服务美利车金融股票代码为ML向美国证监会SEC递交了招股书计划在美国市场IPO融资亿美元但是美利车金融并没有等来它的高光时刻这家主要靠促成汽车贷款赚钱的消费分期服务公司把车开进了内忧外患的漩涡根据知情人士提供的其与警方沟通录音证实公安部统一指挥此次抓捕行动涉及包括深圳武汉辽宁和北京在内的多个办公场所抓捕多名高管当天美利金融北京总部深圳总部武汉分部等多地均遭警方上门调查公司公告称初步确定因警方对关联公司有用分期的调查而引发对美利车金融业务的影响在国内各大网络投诉平台上对于美利车金融的投诉中也能看到套路贷非法催收等严重问题根据第三方投诉平台聚投诉上有不少针对美利车金融的投诉包括砍头息套路贷暴力催收欺诈等黑猫投诉平台上关于美利车金融的投诉也多涉及套路贷砍头息欺诈等多位用户投诉均标明美利车金融关联催收公司涉嫌爆通讯录电话骚扰短信恐吓等软暴力方式等聚投诉上一位名为李红院的用户投诉称我在美利车金融办了车贷业务不注意逾期一天有催收人员打电话过来提醒我还款合情合理但是催收人员上来就扬言限我多少分钟内把钱存到银行卡不然就让我同事知道让我家人亲戚知道并爆出我工作地址家庭地址以及家人亲戚姓名对我进行人格侮辱有消息透露目前公安方面正在调查与美利金融和刘雁南有关联的所有高炮平台寻找借款人做笔录并努力恢复底被这些高炮平台删除的系统数据实际上根据美利车金融的招股说明书来看也说明了美利车的盈利模式主要是资金掮客和催收资料显示上半美利车金融净利润亿元主要通过促成汽车贷款取得收入通常这笔助贷服务费通常占贷款本金的至而资金合作伙伴即银行收取的平均化利率为亿贷款存巨大回笼压力最大资金提供方占比近成但是这一切随着美利车金融被警方查封之后都在快速消散数据显示截至美利车金融汽车融资贷款余额为亿元这亿元存量贷款回笼压力巨大客户回款已经成为迫在眉睫的问题事实上在警方大规模带走美利车金融高层和员工之后用户开始出现拒绝还款的情况从开始这一情况越演越烈美利车唯一的官方客服电话已经无法接通在美利车金融的微信服务号美利车服务号上不到一个时间已经连续公告三次号召用户积极还款从各方透露的信息来看美利车原有的还款路径纷纷失效有知情人士分析称可能是美利车金融自身的相关金融账户或被警方查封导致还款失效多位美利车金融的贷款用户也向券商中国记者反映目前美利车金融APP已无法正常还款为避免造成被动逾期个别用户只能通过银行的还款通道协商还款而目前美利车营业厅已停止运营客服电话也无法打通在安卓商店和苹果商店美利车金融APP已经普遍下架目前美利车金融的唯一官方客服电话已经被停止使用而线下业务全面陷入瘫痪停顿大批业务人员被警方调查问询更多则在惶惶不安之中目前为数众多的美利车金融用户声称遭受套路贷骚扰和暴力催收北京广州郑州等地区由受害人自发建立的各地投诉微信群已有数十个之多正在组织联合投诉举报并以存在合同纠纷为由拒绝还款向各地方法院对美利车金融提起诉讼同时把矛头指向了为美利车金融提供资金的银行有不少车贷人表示美利车金融在他们不知情的情况下在购车贷款合同中被额外添加了一笔借款大部分车贷人都是在即将还清贷款时才发现需要多还万元甚至更多车贷人想按最初申请的贷款金额一次性还清贷款却被美利车金融一口回绝更离谱的是不少车贷人还在继续按时还款的情况下频繁接到暴力催收的骚扰电话和短信目前美利车金融在号召还款的资金方有新网银行微众银行众邦银行南京银行等其中美利车控股第一大股东刘永好旗下的新网银行是美利车金融的最重要的资金方而刘永好旗下的新希望集团是新网银行大股东持股另外新希望集团也是美利车第一大股东持股根据美利车金融招股书披露信息来看美利车金融的资金提供方集中度非常高其中最大的融资合作伙伴提供资金占比为第二名所占比例为在这两大金融机构中新网银行便是其中一家尽管信息没有披露出新网银行是第一大融资来源还是第二大融资来源根据上半美利车金融汽车融资贷款余额数据亿元来看最大的融资提供方这家银行仅对美利车一家即融出亿元市场普遍认为新希望集团同时作为新网银行和美利车金融的第一大股东新网银行成为美利车金融最重要资金合作伙伴不足为奇多位车贷用户向记者反映在美利车金融无法正常还款之后多次接到新网银行催收短信有的甚至在全额还款之后仍然没有拿到抵押的机动车登记证书不仅仅美利车金融亿元贷款回笼压力巨大事实上新网银行面临的资金出具方业务压力也非常大在全国陆续全面取缔PP网贷业务的关口新网银行仍不愿退出网贷资金存管业务甚至还坐着国内最大的PP存管银行名头且在合作中新网银行通常扮演了资金出具方的角色作为一家民营银行目前新网银行资金来源主要有三个渠道一是银行间的拆借市场新网银行已和家银行达成了同业授信和拆借协议二是联合授信已跟家银行签订了联合授信协议三是对已有的资产做成证券化来变现流转显然从这三个渠道来看很大一部分都不属于新网银行自有资金",
            "limit": 2
            }

In [486]:
result = requests.post(url, json.dumps(test_type2_data), headers=header)
json.loads(result.text)

{'246': {'sim_article_contexts': '文 | 欧拉 木一\n去年开始，二车手金融领域发生了翻天覆地的变化。\n2019年7月，优信旗下的二手车金融业务线，被卖给了58集团控股的汽车金融相关企业Golden Pacer。\n知情人士透露：“58并未完全接住优信二手车金融的业务，两个团队磨合有问题，走了可能有一半的人。”\n2019年11月，美利金融的创始人被调查，连带美利金融上市搁浅，业务暂停。\n二手车金融的两大玩家，皆遭遇挫折，而吐出来的市场份额，“正在被另一个巨头——平安收割”。\n金融科技等创新公司在二手车金融市场鏖战三年，最终却是为传统金融做了嫁衣？\n01\xa0平安收割\n二手车金融市场有三大巨头：平安、优信金融和美利金融。\n“三者的市场份额大概是这样的，平安占15%，优信金融和美利金融大概占10%到15%。”汽车金融行业资深人士张帆称，“这三者约占了市场40%的份额。”\n而另外一些市场份额，被工行、地方系等零星分割。\n但这种微妙的平衡，却在去年下半年被彻底打破。\n2019年7月，优信金融将二手车金融业务卖给了58集团控股的汽车金融相关企业Golden Pacer，以换取1亿美元的现金和一定数量的Golden Pacer股份。\n“这是优信不错的一步棋。”张帆认为。\n“优信的二手车金融业务，是管理团队、业务和资产负债一起卖的，实际上，加上逾期和资产的整体价值，可能整个资产包是负的。”张帆称，优信试图据此缓解“艰难的现金流问题”。\n另一方面，剥离了金融属性的优信，“也许能得到资本市场更多的认可，因为过去优信的金融业务太重，估值一直上不去”。\n获得现金流，轻装上阵，优信也许还能和瓜子二手车一战。\n“但被卖掉的优信二手车金融业务，Golden Pacer却并未完全接住。”张帆称，“业务接过去之后，很快就掉下来。”\n一些曾就职于优信二手车金融的Golden Pacer员工透露，今年年初，Golden Pacer开始撤掉一些城市的网点，并优化员工。\n优信二手车金融在被出售后，发展不算顺利，而美利金融则更凄惨。\n2019年11月，美利金融的创始人被调查，连带美利金融上市搁浅，业务暂停。\n“这相当于优信和美利的业务均遭到了打击，现在它们的大部分市场份额被平安吃掉了。”张帆表示，平安成为了最大的赢家，“现在已经占了整

In [464]:
# 不同type召回结果
test_type3_data = {"article_id":9999,
            "status":True,
            "type":"financial",
            "article_title":"一文带你看懂互联网金融产品的分类",
            "article_context":"在互联网时代下 传统金融机构和互联网企业利用互联网技术和信息通信技术 实现新型金融业务模式  互联网金融 近几  互联网金融发展得如火如荼 市场中的互联网金融产品层出不穷 对于刚踏入金融领域的运营小白来说 可能还不太清楚它们的具体分类 下面大家就跟小编一起来了解一下吧   目前我们常见的互联网金融产品可以分为以下几大类   第三方支付类 以支付宝 微信支付为代表  P P网络借贷类 典型代表有 陆金所 人人贷 宜人贷 拍拍贷等  投资理财类 我们经常使用的余额宝 京东金融等  网络众筹类 可以理解为大众筹资或群众筹资 是指以团购预购的形式 向互联网金融网友募集项目资金的模式 在淘宝和京东电商平台上都有众筹板块  虚拟货币类 指的是所有通过互联网技术直接从原理上改造金融的产品 典型的是比特币  一 第三方支付类 第三方支付是指具备一定实力和信誉保障的独立机构 借助计算机通信和信息安全技术提供交易的支持平台 在银行与用户之间建立连接的电子支付模式 近几 第三方支付迅猛发展 已不仅仅局限于最初的互联网支付 而是成为线上线下全面覆盖 应用场景更为丰富的综合支付工具 如今 在我们 常生活的各个领域都能看到第三方支付的身影 人们也越来越习惯于使用第三方支付平台进行支付    艾瑞咨询     中国第三方移动支付报告显示      整体移动支付规模突破   万亿元 是毫无争议的世界第一大移动支付市场 其中 支付宝以      的市场份额遥遥领先 微信支付为       数据直接反映出移动支付市场是的格局 呈现支付宝和微信支付两家独大的局面 用户习惯也已经完全养成 留给其他玩家的空间微乎其微 虽然还有狭窄的细分市场可以抢占流量 但市场格局几乎不存在大幅改变的可能    数据   支付宝和微信支付依托于强大的用户体量优势 让小公司和小产品无法取得突破性的成果 这也是目前支付类产品被垄断的主要原因 因此 第三方移动支付平台未来的发展方向应集中在以下两个方面    找准自身支付平台的定位 各大支付平台应提高支付安全性 强化其支付功能 同时 充分挖掘大数据的优势 创新商业模式 降低对背后支持平台的依赖 实现独立发展    第三方移动支付平台和商业银行之间合作大于竞争 应加强与商业银行的合作 形成互利共赢的支付产业链条   二 P P网络借贷 P P网络借贷搭载互联网平台 将传统借贷行业发展到网络上 以收取一定利息为目的 向其他个人提供小额贷款的金融模式 它的本质是一种债权市场 通过第三方互联网平台进行资金借互联网产品 贷双方的匹配 需要借贷的人群可以通过P P平台寻找到有出借能力并且愿意基于一定条件出借的人群 帮助贷款人通过和其他贷款人一起分担一笔借款额度来分散风险 也帮助借款人在充分比较的信息中选择有吸引力的利率条件 在整个借贷过程中P P网络借贷平台扮演的是一个中间人的角色 对借贷双方的信息沟通 资金流动起到牵线搭桥的作用    从     起 P P以其无抵押担保 撮合速度快 借款利率低的特点吸引各种类型的玩家加入 银行 消费金融公司 小贷公司 互联网公司纷纷扎进这片红海市场 各种模式也是层出不穷 这一 业内网贷平台过千家 呈现井喷式发展 打开任意一个APP应用商城搜索借贷APP都可以让你搜到眼花缭乱   伴随P P网络借贷快速增长 问题不断涌现 平台跑路现象层出不穷 截至      底 能够检测到的P P借贷平台共     家 正常运营仅有     家 仅占     的比例 环比下降            互金专项整治活动以来 新上线平台持续减少 问题平台数量维持高位    造成P P网络借贷平台形势一度混乱的原因是 一方面 国内整体征信体系缺乏有效监管 另一方面 互联网评估借款人的还款能力和还款意愿非常难 还要时刻防止虚假骗贷 再加上互联网对违约和逾期用户的催收更是鞭长莫及   因此 P P网络借贷要获得良好地发展 就必须具备较强的信息甄别能力 能够对个体商户或小微企业建立起良好的信用评价体系 为有意愿贷款者筛选出具有良好信用的借款者 从而减少P P网络贷款的信用风险   三 投资理财 投资理财主要是指投资者通过合理安排资金 运用诸如 储蓄 债券 基金 股票 保险等投资理财工具对个人 家庭及企业资产进行管理和分配 以达到保值增值目的的行为    随着金融科技的发展 财富管理行业也逐步进入崭新的时代 互联网理财产品应运而生 据了解 如今的理财APP已达上千种 BAT等巨头也各自推出理财软件 阿里的余额宝 腾讯的腾讯财富通 百度旗下的百度理财 甚至还有工商银行手机客户端品牌全新升级 融e行 等   多样化的互联网理财产品进入大众视线 传统理财逐渐被互联网理财替代 互联网理财成为国民理财的主要途径 归其原因 一是因为理财观念渐渐普及大众 大众理财规模也越来越大 二是互联网理财高收益 低风险 方便快捷和高流动性等优点迎合大众理财需求 三是互联网理财体验更佳 信息获取成本大幅下降    正因此 互联网理财自诞生以来一路高歌猛进      国家金融与发展实验室联合腾讯金融科技智库共同发布 中显示 我国互联网理财指数已经由     的   点增长到     的   点 四 时间内增长了近 倍 此外  预计到     底互联网理财规模将达到    万亿 到     或将达到    万亿 在居民理财渠道中占据重要地位   虽然互联网理财发展迅猛 但风险切不可轻视 由于互联网理财市场门槛低 用户的整体素质参差不齐 市场监管体系还不完善 跑路 倒闭等违法违规事件时常爆发 导致行业争议不断 所以投资者进行理财时应选择正规 有银行存管 运营稳健的平台 在自身风险承受范围内追求合理的收益   不过 为促进互联网金融行业健康发展 近几  国家政府开始大力整顿 推动互联网金融合规合法发展 可以预见 强监管将带来的明显挤出效应 很多不合规平台会被清除 使行业收益率回归合理水平 互联网理财行业将进一步规范化发展 前景仍可期   四 网络众筹 众筹 也可称做大众筹资 是指在互联网上利用团购或预购的方式 募集项目筹措资金的模式 利用互联网传播途径 小企业或个人可通过众筹的方式面对公众 展示创意 争取大家的支持 获得资金援助 相对于传统的融资方式 众筹更为开放 能否获得资金也不再是由项目的商业价值作为唯一标准 使得任何有想法有创意的人都能可以通过众筹方式获得项目启动的第一笔资金 为更多创业者提供无限可能    自     以来 以创投圈 天使汇为代表的一批针对种子期 天使期的创业服务平台 以一种 众投 的模式进入的人们的视野 并很好的承接了对众筹本意的理解 但是与热闹的P P相对 众筹行业遇冷似乎才是近 来业内外的共识 这主要是由于国内对公开募资的规定不明确 容易踩到非法集资的红线 以及众筹项目优劣评判困难 回报率的不确定性等原因 导致投资者的投资意愿较低 涉及金额也相对较小 无法形成一定的客户规模   据众筹家人创咨询统计 截至       底 全国共上线过众筹平台   家 仍然正常运营的仅剩   家     上半 共有     个众筹项目 其中成功项目有     个 占比       成功项目实际融资额达      亿元    从平台数量上来看 高速成长的互联网环境并没有提升众筹行业的整体规模 反而经历了从高速增长到极速降落的过程 呈现出一窝蜂的兴起 而又一大片的倒下的局面 侧面反映出众筹在野蛮生长后的冷静思考 应该避免盲目扩张 运营要转向精细化 要体现出自身的差异化 凸显出自身的垂直化特征 从项目数量和融资额上看 众筹行业整体态势乐观 但报告也指出 其增长主要来源于物权型众筹和权益型众筹   五 虚拟货币 作为虚拟货币的载体 在互联网存在后 虚拟货币也应运而生 虚拟货币区别于真实货币的最大特征是在虚拟网络中流通 同时又具有自身特定的货币价值 这样的特征使得虚拟货币的价值局限在网络系统 而不能等同于实际货币在现实社会中广泛流通使用 从某种意义上来说虚拟货币比以往任何的互联网金融形式都更具有颠覆性    我国的虚拟货币相对于国外 发展较晚 时间也比较短 但发展相当迅速 种类繁多 五花八门 第一类是大家熟悉的游戏币 第二类是门户网站或者即时通讯工具服务商发行的专用货币 第三类互联网上的虚拟货币 如比特币 莱特币等   随着 互联网  与生产 消费等领域的融合逐步加深 使得虚拟货币的流通范围进一步扩大 加强了虚拟货币的一般等价物的货币特性 虚拟货币的货币职能逐步完善 同时 由于虚拟货币发行 流通成本微小 发行商与消费者使用虚拟货币的意愿加强 这些都使得虚拟货币规模不断增加 据统计 我国每 的虚拟货币规模已经达到了几十亿元的级别 并且仍以每        的速度增长   在 互联网  背景下 虚拟货币必将迎来更加迅猛的发展 虚拟货币的应用范围会越来越广阔 给整个社会带来方方面面不可忽视的影响 但在此过程中 安全问题也如影随形 虚拟货币被盗的事件屡屡发生 这就需要我们掌握尽可能的避免灾害的知识和技能 同时 国家也应加强对网络虚拟货币的规范和监管 形成更成熟和完善的体系 保证其长久稳定发展 如此 我们才能成为互联网金融时代的受益者   除此之外 互联网金融产品还包括基金 保险 证券等等。互联网产品",
            "limit": 2
            }

In [465]:
result = requests.post(url, json.dumps(test_type3_data), headers=header)
json.loads(result.text)

{'496': {'sim_article_contexts': '法国哲学家鲍德里亚认为，生产社会发展到消费社会，从物的消费发展到符号消费，商品不再是“需要”而是“想要”。当下，中国社会已经进入到生产过剩的时代，由此刺激了消费主义的形成。\n人们通过对消费的追求来实现自我，表达生活方式，确定身份、地位、寻找自己人生的意义。通过消费来区隔身份，来构建出一种想象性的自我认同。\n过去的2018年，在实体经济面临下行压力的大背景下，为了拉动国家经济增长，提升人们生活水平，政府亦通过减免税收，提升社会保障机制来全力支持消费。\n从央行公布的短期消费贷款数据可以看出，2018年消费贷款余额仍然保持快速增长，下半年相对上半年有所回落，但也在30%左右。\n不过，消费金融各大参与主体的势力范围已悄然生变。\n近年来，商业银行在加速零售业务转型，大力发展短期个人贷款及信用卡业务。实际上，不仅仅是商业银行。面对万亿元规模的消费金融市场，持牌消费金融公司、互联网金融平台、金融科技公司等机构都欲从中分羹。\n2018年，持牌消费金融公司增资潮涌。全年共有10家持牌机构拟增资或完成增资，增资额超过88亿元。仅12月一个月，招联消费金融、长银消费金融宣布分别拟从28.59亿元、3.6亿元增资至48.59亿元、10.5亿元。\n特别是，部分渠道/厂商平台等跨界玩家也开始跑步入场。典型的代表分别有今日头条、美团、携程、滴滴等，这类平台有内生流量，可与场景、服务、产业链进行深度整合。\n相比之下，P2P平台消费金融业务式微，专业服务平台也渐趋艰难。融360大数据研究院李万赋表示，尽管消费金融类资产成交量占比越来越高，受政策影响，P2P平台的业务增长受到抑制，拖累消费金融业务，消费金融的成交额持续下跌，未来的生存会越来越困难。\n此外，分期服务平台同样面临P2P平台一样的资产困境，风控能力甚至低于P2P平台，导致诈骗事件频发，一度被推上风口浪尖，失败的平台案例不在少数。\n信用卡应偿额飙升\n从规模上看，传统商业银行仍然是我国消费金融领域的王者，其优势地位得到了进一步巩固。\n统计数据表明，2014-2018年，零售新增贷款分别为3.29万亿、3.87万亿、6.33万亿、7.13万亿、6.91万亿（1-11月）。2016-2017年的高供给主要是住房按揭，但到了地产调控的2018年，前11个月的供

### Reference
[gensim doc2vec tutorial](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py)